In [2]:
import time
import statistics as stats
import multiprocessing
import time 
import random
import time
from tqdm import tqdm
from datetime import timedelta
from datetime import datetime
start_program = time.time()
from multiprocessing import *
import pandas as pd
import numpy as np
from threading import Thread
import time
import subprocess 


In [3]:
results = [[] for i in range(5)]

n_it = 100


n_st = number_stations = 10
n_s = number_sensors = 100

def_st = 1
def_s = 3
def_r = 'day'


set_date = [random.random() for i in range(500)]


In [4]:

def str_time_prop(start, end, time_format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))



def random_date(start, end, prop, dform = '%Y-%m-%dT%H:%M:%S'):
    return str_time_prop(start, end, dform, prop)
    
    
def to_pm(v):
    return str(round(v[0][0],2)) + "$" + '\\' + "pm$" + str(round(v[1][0],2))


# Queries

In [5]:
# A simple class
# attribute
# e_q1 = """select seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, s<sid>@tt FROM d1_v WHERE id_station = 'st<stid>';"""
e_q1 = """select id_station, seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, <sid> FROM d1_v WHERE id_station in <stid>;"""


e_q2 = """select id_station, tfe, <sidlist> from (
select id_station, seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, <sfilter> as fe, t@fe as tfe, <sid_filtered> FROM d1_v WHERE id_station in <stid>
); """ #!seq_filter_search(s<sid>@tt > 0.95, tt),. <sid>@fe 

e_q2AND = """select id_station, seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, <sfilterAND> as fe, <sid_filtered> FROM d1_v WHERE id_station in <stid>; """ #!seq_filter_search(s<sid>@tt > 0.95, tt),. <sid>@fe 


e_q3 = """SELECT id_station, ! seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, <seq_avg> FROM d1_v WHERE id_station in <stid>;""" # where id_station = 'st<stid>', <seq_avg>(<sid>@tt)


e_q4 = """select id_station, ts, <sidlist> from (
select id_station, seq_search(t,<timestamp> - <nb> * <rangesUnit>, <timestamp>) as tt, seq_group_agg_avg(t@tt , t@tt/3600) as ts, <seq_group_agg_avg> FROM d1_v where id_station in <stid>
);""" #seq_group_agg_avg(s<sid>@tt, t@tt/3600) 


e_q5 = """select seq_aprogres_datetime(<timestamp> -  <nb> * <rangesUnit>, 5, <nb> * <rangesUnit>) as ts5, <seq_stretch> from d1_v where id_station in <stid>;""" #seq_stretch(ts5,t,s<sid>)



In [8]:
import pymonetdb
from tqdm import tqdm 
import time

class EXtremeDB:
 
    # A sample method 
    @staticmethod
    def query(query, db, max_d, rangesUnit, n_it, start_date = "2019-04-01 00:00:00", stop_date = "2019-04-30 00:00:00", n_st = 1, n_s = 10):
        
        import exdb 
        import datetime
        exdb.init_runtime(debug = False, shm = False, disk = False, tmgr = 'mursiw')
        con = exdb.connect('diufrm118', 5001)
        curs = con.cursor()
        curs.execute("SELECT s23 FROM d1_v where id_station = 'st3'")
        curs.fetchall()
        results = [[],[]]
        options = {"day" : 60 * 60* 24,
                   "week" : 60 * 60* 24 * 7,
                   "minute" : 60,
                   "hour" : 60 * 60,
                   "second" : 1,
                   "month" : 60 * 60 * 24 * 30,
                   "year" :  60 * 60 * 24 * 30 * 12
        }
        duration = max_d
        runtimes = []
        full_time = time.time()
        for i in tqdm(range(n_it)):
#             time.sleep(0.5)
            date = random_date(start_date, stop_date, set_date[(duration*i)%500], dform = '%Y-%m-%d %H:%M:%S')
            date = int(time.mktime(datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S').timetuple()))
            temp = query.replace("<timestamp>", str(date))
            temp = temp.replace("<nb>", str(duration))
#             temp = temp.replace("<sid>", str(set_s[(duration*i)%500]))
            temp = temp.replace("<rangesUnit>", str(options[rangesUnit]))
        
            li = ['st' + str(z) for z in random.sample(range(number_stations), n_st)]
            q = '(' + "'" + li[0] + "'"
            for j in li[1:]:
                q += ', ' + "'" + j + "'"
            q += ")"
            temp = temp.replace("<stid>", q)
            temp = temp.replace("<db>", db)
            
            # sensor
            rand = [str(z) for z in random.sample(range(number_sensors), n_s)]
#             print(rand)
            sidlist = 's' + rand[0]
            for j in rand[1:]:
                sidlist += ',' + 's' +  j
#             print(sidlist)
            li = ['s' + str(z) + "@tt" for z in rand]
            li_filtered = ['s' + str(z) + "@fe as s" + str(z) for z in rand]
#             <seq_group_agg_avg> seq_group_agg_avg(s<sid>@tt, t@tt/3600), 
#             <seq_stretch> #seq_stretch(ts5,t,s<sid>), 
#             <sfilter> !seq_filter_search(s<sid>@tt > 0.95, tt)
            
            q = li[0]
            q_filtered = li_filtered[0] 
            q_seq_group_agg_avg = "seq_group_agg_avg(" + li[0] + " , t@tt/3600) as " + li[0].split('@')[0]
            q_seq_avg = "seq_avg(" + li[0] + ")" 
            q_seq_stretch = "seq_stretch(ts5,t," + li[0].split('@')[0] + ")" 
#             q_filter = "!seq_filter_search(" +li[0] + "> 0.95, tt)"
            q_filter = "!seq_filter_search(" +li[0] + "> 0.95"
            q_filterAND = "!seq_filter_search(" +li[0] + "> 0.95"
            
#             q_avg = 'avg(' + li[0] + ')'
            for j in range(1,len(li_filtered)):
                q_filtered += ', ' + li_filtered[j] 
    
            for j in li[1:]:
                q += ', ' + j
                q_seq_avg += ", seq_avg(" + j + ")" 
                q_seq_group_agg_avg += ", seq_group_agg_avg(" + j + " , t@tt/3600)" + " as " +  j.split('@')[0] #        li[0] + ' > 0.95'
                q_seq_stretch += ", seq_stretch(ts5,t," + j.split('@')[0] + ")" 
# #                 q_filter += ", !seq_filter_search(" + j + "> 0.95, tt)"
#                 q_filter += " OR " + j + "> 0.95"
#                 q_filterAND += " AND " + j + "> 0.95"
# #                 q_filter += ' OR ' + j + ' > 0.95'
# #                 q_avg += ', ' + 'avg(' + j + ')'
            temp = temp.replace("<sid>", q)
            temp = temp.replace("<sidlist>", sidlist)
            temp = temp.replace("<seq_avg>", q_seq_avg)
            temp = temp.replace("<sid_filtered>", q_filtered)
            temp = temp.replace("<seq_group_agg_avg>", q_seq_group_agg_avg)
            temp = temp.replace("<sfilter>", q_filter + ", tt)")
            temp = temp.replace("<sfilterAND>", q_filterAND + ", tt)")
            temp = temp.replace("<seq_stretch>", q_seq_stretch)
#             temp = temp.replace("<avg_s>", q_avg)            
        
#             print(temp)
            start = time.time()
            curs.execute(temp)
            curs.fetchall()
            # print(curs.rowcount)
#             print(curs.fetchall()[0])
            diff = (time.time()-start)*1000
#                 print(temp, diff)
            runtimes.append(diff)
            if time.time() - full_time > 20 and i > 5: 
                break  
#         print(runtimes)
#         print(temp)
        results[0].append(stats.mean(runtimes))
#             results[1].append(percentile(runtimes,85))
        results[1].append(stats.stdev(runtimes))
        con.close()
        return results

    
    
    @staticmethod
    def insert_file(batch_size,rate, st):
#         template_sql = "INSERT INTO d1_wide_slow_ingest (time, id_station"
#         for i in range(100):
#             template_sql += ", s" + str(i)
#         template_sql += ") VALUES ('%s', '%s',"  + ','.join(100* [' %s']) + ');'

        import exdb 
        import datetime
        exdb.init_runtime(debug = False, shm = False, disk = False, tmgr = 'mursiw')
        conn = exdb.connect('diufrm118', 5001)
        cur = conn.cursor()
        cur.execute("set append_mode true")
        file = '/home/abdel/venv2/bin/insertion-100/short_ts' + str(st) + '.csv'
        f = open(file, "r")
        f = f.read()
        f = f.splitlines()[1:]
        for i in range(len(f)):
            f[i] = f[i].split(',')            
            temp = "'" + f[i][1] + "' , " + "'" + ' '.join(f[i][0].split('T')) + "' "
            for e in f[i][2:]:
                temp += ", " + str(e)
            f[i] = temp
        queries = []
        q_template = "INSERT or UPDATE INTO d1_ingest_v (id_station, t"
        for k in range(100):
            q_template += ", s" + str(k)
        q_template += ") VALUES ("
        for i in range(0, len(f), rate):
            q = q_template +  f[i] + ")"
            for i in f[i+1:i+rate]:
                q += ", \n(" + i + ")"
            queries.append(q)
        start = time.time()
        a = 0
        print('insertion st' + str(st) + ' started')
        while not stop_insertion[st]: 
            starttime = time.time()
            cur.execute(queries[a])
            a += 1
            diff = 1.0 - ((time.time() - starttime))
            try: 
                time.sleep(diff)
            except: 
                print(diff)
                
    @staticmethod
    def insert_file_1station(batch_size,rate, st, f):
#         template_sql = "INSERT INTO d1_wide_slow_ingest (time, id_station"
#         for i in range(100):
#             template_sql += ", s" + str(i)
#         template_sql += ") VALUES ('%s', '%s',"  + ','.join(100* [' %s']) + ');'

        import exdb 
        import datetime
        exdb.init_runtime(debug = False, shm = False, disk = False, tmgr = 'mursiw')
        conn = exdb.connect('diufrm118', 5001)
        cur = conn.cursor()
        cur.execute("set append_mode true")
#         file = '/home/abdel/venv2/bin/insertion-100/short_ts' + str(st) + '.csv'
#         f = open(file, "r")
#         f = f.read()
#         f = f.splitlines()[1:]
#         for i in range(len(f)):
#             f[i] = f[i].split(',')            
#             temp = "'" + f[i][1] + "' , " + "'" + ' '.join(f[i][0].split('T')) + "' "
#             for e in f[i][2:]:
#                 temp += ", " + str(e)
#             f[i] = temp
        queries = []
        q_template = "INSERT or UPDATE INTO d1_ingest_v (id_station, t"
        for k in range(100):
            q_template += ", s" + str(k)
        q_template += ") VALUES ("
        for i in range(0, len(f), rate):
            q = q_template +  f[i] + ")"
            for i in f[i+1:i+rate]:
                q += ", \n(" + i + ")"
            queries.append(q)
        start = time.time()
        a = 0
        print('insertion st' + str(st) + ' started')
        while not stop_insertion[st]: 
            starttime = time.time()
            cur.execute(queries[a])
            a += 1
            diff = 1.0 - ((time.time() - starttime))
            try: 
                time.sleep(diff)
            except: 
                print(diff)
        
    @staticmethod
    def run_all(start, stop, db):
        results[0].append(to_pm(EXtremeDB.query(e_q1, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[1].append(to_pm(EXtremeDB.query(e_q2, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[2].append(to_pm(EXtremeDB.query(e_q3, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[3].append(to_pm(EXtremeDB.query(e_q4, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[4].append(to_pm(EXtremeDB.query(e_q5, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
    
    
    
    

In [ ]:
EXtremeDB.run_all("2019-04-01 00:00:00", "2019-04-30 00:00:00", "d1_v")


In [6]:
data = []
for st in tqdm(range(10)):    
    file = '/home/abdel/venv2/bin/insertion-100/short_ts' + str(st) + '.csv'
    f = open(file, "r")
    f = f.read()
    f = f.splitlines()[1:]
    for i in range(len(f)):
        f[i] = f[i].split(',')            
        temp = "'" + f[i][1] + "' , " + "'" + ' '.join(f[i][0].split('T')) + "' "
        for e in f[i][2:]:
            temp += ", " + str(e)
        f[i] = temp
    data.append(f)            
data2 = []
for i in range(len(data[0])):
    for j in range(10):
        data2.append(data[j][i])
f = data2            
            

100%|████████████████████████████████████████████████████████████████████| 10/10 [01:44<00:00, 10.49s/it]


In [13]:
n_threads = 1
stop_insertion = [False for i in range(n_threads)]
rate = 20

# 20K dp/s

# Launch insertion
for i in range(n_threads):  
    t1 = Thread(target = EXtremeDB.insert_file_1station, args=(100_000, rate, i, f))
#     t1 = Thread(target = ClickHouse.insert_by_point, args=(rate, i))
    t1.setDaemon(True)
    t1.start()

# insert_new(200, 1)

insertion st0 started
-4.012300491333008
-1.452418327331543
-2.456230640411377
-0.2188127040863037
-1.896813154220581
-4.409588575363159
-1.06394362449646
-0.2858297824859619
-1.2537846565246582
-0.0810401439666748
-2.1417651176452637
-1.4597065448760986
-2.5497210025787354
-0.6682639122009277
-0.09005117416381836
-1.0354056358337402
-0.35732436180114746
-1.389714002609253
-0.8211641311645508
-1.9855656623840332
-1.3148953914642334
-0.13666462898254395
-0.6539778709411621
-2.5592103004455566
-2.98258113861084
-0.3541383743286133
-0.23531222343444824
-2.859497547149658
-0.1232752799987793
-0.42272114753723145
-1.8901925086975098
-1.4022793769836426
-3.025468111038208
-1.3633465766906738
-0.8936707973480225
-3.3665120601654053
-0.5167779922485352
-3.8632240295410156
-1.071958303451538
-0.6986246109008789
-0.453446626663208
-3.3067421913146973
-2.2617478370666504
-0.9629368782043457
-0.8115174770355225
-0.5018010139465332
-2.0719809532165527
-1.4050133228302002
-0.0635068416595459
-2.9749

-0.6123571395874023
-2.8227579593658447
-2.0101990699768066
-0.25349974632263184
-3.510791301727295
-2.48115611076355
-4.323991775512695
-3.1576485633850098
-4.1503660678863525
-0.07856297492980957
-1.2528331279754639
-2.2491443157196045
-1.1108002662658691
-0.1283564567565918
-1.213630199432373
-2.240945339202881
-3.7918708324432373
-2.886457681655884
-3.4021785259246826
-2.123504638671875
-0.6208407878875732
-2.044989585876465
-1.0192177295684814
-0.4087064266204834
-1.282991647720337
-1.4916982650756836
-0.460040807723999
-0.508371114730835
-0.2969489097595215
-1.0130209922790527
-0.19140958786010742
-1.1759662628173828
-0.0803067684173584
-2.0414624214172363
-1.4173774719238281
-2.089573621749878
-0.022845745086669922
-0.7030658721923828
-4.405097484588623
-0.19614219665527344
-0.16545772552490234
-0.7691340446472168
-2.6026034355163574
-0.7998991012573242
-0.347048282623291
-0.6066832542419434
-1.08363676071167
-3.276146173477173
-0.9168550968170166
-0.6018064022064209
-2.84251308

-0.24031329154968262
-2.074523687362671
-3.223849058151245
-0.28614377975463867
-1.9961836338043213
-3.239450216293335
-1.5063893795013428
-0.264998197555542
-0.06548810005187988
-2.176501512527466
-2.8378419876098633
-2.6316051483154297
-3.1321773529052734
-3.2074835300445557
-0.5275213718414307
-0.8309082984924316
-2.9312493801116943
-2.0824849605560303
-0.600886344909668
-1.623020887374878
-0.9311330318450928
-1.8365709781646729
-1.9823133945465088
-0.26387667655944824
-1.906479835510254
-1.017796277999878
-0.15033936500549316
-2.1393179893493652
-1.8740580081939697
-1.0486993789672852
-1.016270399093628
-1.212146282196045
-0.8952126502990723
-2.3909995555877686
-1.927351951599121
-0.8172144889831543
-0.08317017555236816
-1.0357863903045654
-1.9030299186706543
-1.0607128143310547
-1.5871639251708984
-1.0677213668823242
-0.4883086681365967
-1.71694016456604
-0.23507475852966309
-0.764507532119751
-1.3889100551605225
-0.586857795715332
-1.3953187465667725
-0.9100005626678467
-0.457313

-2.3157901763916016
-0.5340120792388916
-2.0762462615966797
-0.14908385276794434
-2.174217700958252
-0.24536895751953125
-1.2153239250183105
-2.4740402698516846
-0.5965077877044678
-1.597684621810913
-0.256988525390625
-2.389677047729492
-0.463850736618042
-1.1384015083312988
-1.8425779342651367
-0.26444506645202637
-1.7540245056152344
-0.1947314739227295
-1.698411226272583
-0.11718416213989258
-0.05161404609680176
-1.1564643383026123
-1.0196723937988281
-1.1188693046569824
-2.499750852584839
-1.0737853050231934
-2.0859768390655518
-1.8991672992706299
-2.594695806503296
-2.1020586490631104
-1.706723690032959
-0.5682961940765381
-1.574493408203125
-0.5669209957122803
-1.9123587608337402
-0.421356201171875
-1.5695512294769287
-0.30408215522766113
-1.886687994003296
-0.5760498046875
-1.29244065284729
-0.6266777515411377
-1.2138960361480713
-2.1235427856445312
-2.1817750930786133
-1.1714584827423096
-0.9466640949249268
-2.1828293800354004
-2.828634738922119
-1.9398953914642334
-0.738237857

-1.0484042167663574
-2.1567368507385254
-4.032689094543457
-0.22437286376953125
-0.0660409927368164
-0.9436171054840088
-1.160926103591919
-0.328627347946167
-2.7026538848876953
-2.5702223777770996
-0.14903974533081055
-0.3541574478149414
-1.6730537414550781
-2.0191781520843506
-0.004563093185424805
-0.7387635707855225
-0.26397228240966797
-1.904935598373413
-1.4412128925323486
-2.1818811893463135
-1.9235029220581055
-2.9336414337158203
-2.932236433029175
-1.827782154083252
-3.0609982013702393
-2.0491600036621094
-1.2720680236816406
-0.6198651790618896
-1.3608851432800293
-2.3776235580444336
-2.8863935470581055
-2.36078143119812
-1.56546950340271
-0.5343472957611084
-0.35506653785705566
-1.6435329914093018
-1.55279541015625
-0.4125397205352783
-1.7369933128356934
-0.6237921714782715
-1.2468845844268799
-0.7808551788330078
-0.06777834892272949
-0.0640263557434082
-1.9037151336669922
-0.8341882228851318
-0.0031723976135253906
-0.9213442802429199
-3.015371322631836
-1.6917803287506104
-0.

...

-1.9963059425354004
-2.883261203765869
-1.8622605800628662
-2.2946202754974365
-0.5438857078552246
-1.255424976348877
-1.7993535995483398
-0.03247499465942383
-2.09820294380188
-0.08832621574401855
-1.4238479137420654
-3.15303111076355
-2.576221227645874
-0.8485865592956543
-1.9648349285125732
-2.7545325756073
-0.8538417816162109
-1.667595386505127
-1.2915284633636475
-0.4907267093658447
-1.5829532146453857
-0.8325989246368408
-1.46382737159729
-0.6682579517364502
-0.47530484199523926
-0.32212376594543457
-2.239307403564453
-1.9989244937896729
-0.6678647994995117
-1.3161876201629639
-0.6098182201385498
-2.3481051921844482
-0.6700136661529541
-0.22249078750610352
-0.9494976997375488
-0.9614760875701904
-1.1232342720031738
-2.5784761905670166
-0.8267722129821777
-1.2609412670135498
-1.1445977687835693
-1.8092420101165771
-0.11449742317199707
-1.7509076595306396
-0.42249464988708496
-1.5924255847930908
-1.3374202251434326
-0.2464733123779297
-2.0752923488616943
-0.19392633438110352
-2.351

-2.571100950241089
-2.5345396995544434
-0.22291016578674316
-1.4699633121490479
-0.3065526485443115
-1.7292137145996094
-0.25890445709228516
-0.1811695098876953
-2.4257168769836426
-3.1882331371307373
-2.0220139026641846
-3.0192134380340576
-2.0732991695404053
-0.8857712745666504
-1.3249244689941406
-0.8905065059661865
-1.4626119136810303
-0.802295446395874
-2.263519048690796
-1.0452296733856201
-0.5940806865692139
-1.459118127822876
-1.642906904220581
-0.5927002429962158
-1.54756498336792
-0.46241331100463867
-1.454639196395874
-1.8583340644836426
-2.090656042098999
-0.19613099098205566
-1.6322641372680664
-0.22234272956848145
-1.3148002624511719
-1.1917836666107178
-1.89811372756958
-3.089020013809204
-0.6130383014678955
-0.37180471420288086
-0.9246835708618164
-2.2897531986236572
-1.7494547367095947
-0.765486478805542
-1.6988933086395264
-0.6229534149169922
-2.276576042175293
-0.5175361633300781
-1.4019417762756348
-1.264284610748291
-0.6421365737915039
-1.5138163566589355
-0.194989

-0.3722848892211914
-1.3151767253875732
-0.389864444732666
-0.5987091064453125
-1.639606237411499
-2.285318613052368
-0.22071480751037598
-2.210461139678955
-3.006869316101074
-3.3325672149658203
-2.478633403778076
-2.048245906829834
-1.0041303634643555
-1.1948559284210205
-0.9126725196838379
-1.366821050643921
-0.8557844161987305
-2.3380520343780518
-2.859529972076416
-0.9398202896118164
-3.2210988998413086
-1.3466596603393555
-3.029355764389038
-2.1680185794830322
-0.8677935600280762
-1.330571174621582
-2.701178550720215
-0.0054836273193359375
-1.0153024196624756
-1.2097375392913818
-0.8814070224761963
-1.057558298110962
-0.929081916809082
-1.011613130569458
-2.0977718830108643
-2.1834254264831543
-0.8341293334960938
-0.5307612419128418
-0.9535553455352783
-0.4149627685546875
-1.3804614543914795
-1.8232924938201904
-0.8039100170135498
-2.281458854675293
-1.8173980712890625
-0.3137972354888916
-2.9135100841522217
-2.4928441047668457
-0.9186668395996094
-0.2508556842803955
-1.103643417

-1.7607574462890625
-1.9697391986846924
-0.3372318744659424
-1.955531120300293
-0.8572173118591309
-1.4592573642730713
-2.007117986679077
-0.19165897369384766
-0.9366424083709717
-1.2445330619812012
-2.1866455078125
-0.0037059783935546875
-1.1899902820587158
-1.2119090557098389
-0.26248645782470703
-1.8754925727844238
-0.8600063323974609
-2.662841796875
-2.567467451095581
-0.3858604431152344
-2.036956787109375
-0.2979717254638672
-0.17042040824890137
-0.5476188659667969
-0.8241498470306396
-0.558438777923584
-3.01086163520813
-3.0772156715393066
-0.46790432929992676
-0.22187161445617676
-0.09975481033325195
-2.4435811042785645
-0.9551656246185303
-1.1600537300109863
-3.1571531295776367
-3.482218027114868
-1.3327751159667969
-0.9153594970703125
-1.1759705543518066
-0.8149759769439697
-2.766936779022217
-3.319075345993042
-0.8736944198608398
-1.6866233348846436
-2.050297260284424
-0.18389272689819336
-0.7374567985534668
-0.5083684921264648
-0.39600157737731934
-1.8273417949676514
-0.3441

-0.08600616455078125
-1.0417335033416748
-0.1348428726196289
-0.21178627014160156
-0.20752382278442383
-0.9304518699645996
-0.9209654331207275
-0.06690835952758789
-0.37343263626098633
-2.0767321586608887
-0.817816972732544
-2.7769789695739746
-1.0685772895812988
-1.7489242553710938
-0.5134603977203369
-1.6194572448730469
-0.2913336753845215
-0.3567934036254883
-2.3543965816497803
-2.0335144996643066
-1.9328193664550781
-0.19084835052490234
-1.8683421611785889
-0.04607200622558594
-2.5072765350341797
-3.2993195056915283
-0.11730790138244629
-2.176110029220581
-0.15821194648742676
-2.440438985824585
-3.238976240158081
-1.8061857223510742
-3.150463342666626
-1.983635425567627
-0.6056194305419922
-3.0256383419036865
-1.1410322189331055
-0.6196050643920898
-0.8825249671936035
-1.4270083904266357
-0.8431446552276611
-1.4499237537384033
-0.93684983253479
-1.1715795993804932
-1.8818483352661133
-0.017839908599853516
-1.3458547592163086
-0.9498898983001709
-1.0993051528930664
-0.19692158699035

In [ ]:
n_threads = 10
stop_insertion = [False for i in range(n_threads)]
rate = 20

# 20K dp/s

# Launch insertion
for i in range(n_threads):  
    t1 = Thread(target = EXtremeDB.insert_file, args=(100_000, rate, i))
#     t1 = Thread(target = ClickHouse.insert_by_point, args=(rate, i))
    t1.setDaemon(True)
    t1.start()

# insert_new(200, 1)

In [ ]:
time.sleep(3 * 8640 // rate)

In [ ]:
EXtremeDB.run_all("2019-04-01T00:00:00", "2019-04-30T00:00:00", "d1_v_ingest")

In [ ]:
EXtremeDB.run_all("2019-05-01T01:00:00", "2019-05-02T00:00:00", "d1_v_ingest")

In [ ]:
EXtremeDB.run_all("2019-04-29T01:00:00", "2019-05-02T00:00:00", "d1_v_ingest")

In [12]:
stop_insertion = [True for i in range(n_threads)]


In [ ]:
results 

In [ ]:
runtimes = [[float(results[i][j].split('$')[0]) for j in range(len(results[i]))][:4] for i in range(5)]
variance = [[float(results[i][j].split('$')[2]) for j in range(len(results[i]))][:4] for i in range(5)]

In [ ]:
runtimes = list(map(list, zip(*runtimes)))
variance = list(map(list, zip(*variance)))


In [ ]:
runtimes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
X = np.arange(5)
labels = ['query'+str(1+i) for i in range(5)]
fig = plt.figure(figsize=(8, 6))
ax = fig.add_axes([0,0,1,1])
for i in range(4):
    ax.bar(X +0.15*i, runtimes[i], yerr = variance[i], width = 0.15)
ax.legend(['offline', 'online old', 'online new', 'online mixed', ])
ax.set_xticklabels(labels)
ax.set_xticks(np.arange(len(labels)))
ax.yaxis.grid(True)
plt.title('ClickHouse offline vs. online, rate: '+ str(rate) + ' x ' + str(n_threads))
plt.tight_layout()

# ax.bar(X+0.25, data[1], color = 'g', width = 0.25)
# ax.bar(X+0.5, data[2], color = 'r', width = 0.25)

In [1]:
import exdb 
import datetime
exdb.init_runtime(debug = False, shm = False, disk = False, tmgr = 'mursiw')
con = exdb.connect('diufrm118', 5001)
curs = con.cursor()
curs.execute("DROP table d1_ingest_v;")
curs.execute("CREATE TABLE d1_ingest_v (id_station string PRIMARY KEY, t sequence(TIMESTAMP asc), s0 sequence(DOUBLE), s1 sequence(DOUBLE), s2 sequence(DOUBLE), s3 sequence(DOUBLE), s4 sequence(DOUBLE), s5 sequence(DOUBLE), s6 sequence(DOUBLE), s7 sequence(DOUBLE), s8 sequence(DOUBLE), s9 sequence(DOUBLE), s10 sequence(DOUBLE), s11 sequence(DOUBLE), s12 sequence(DOUBLE), s13 sequence(DOUBLE), s14 sequence(DOUBLE), s15 sequence(DOUBLE), s16 sequence(DOUBLE), s17 sequence(DOUBLE), s18 sequence(DOUBLE), s19 sequence(DOUBLE), s20 sequence(DOUBLE), s21 sequence(DOUBLE), s22 sequence(DOUBLE), s23 sequence(DOUBLE), s24 sequence(DOUBLE), s25 sequence(DOUBLE), s26 sequence(DOUBLE), s27 sequence(DOUBLE), s28 sequence(DOUBLE), s29 sequence(DOUBLE), s30 sequence(DOUBLE), s31 sequence(DOUBLE), s32 sequence(DOUBLE), s33 sequence(DOUBLE), s34 sequence(DOUBLE), s35 sequence(DOUBLE), s36 sequence(DOUBLE), s37 sequence(DOUBLE), s38 sequence(DOUBLE), s39 sequence(DOUBLE), s40 sequence(DOUBLE), s41 sequence(DOUBLE), s42 sequence(DOUBLE), s43 sequence(DOUBLE), s44 sequence(DOUBLE), s45 sequence(DOUBLE), s46 sequence(DOUBLE), s47 sequence(DOUBLE), s48 sequence(DOUBLE), s49 sequence(DOUBLE), s50 sequence(DOUBLE), s51 sequence(DOUBLE), s52 sequence(DOUBLE), s53 sequence(DOUBLE), s54 sequence(DOUBLE), s55 sequence(DOUBLE), s56 sequence(DOUBLE), s57 sequence(DOUBLE), s58 sequence(DOUBLE), s59 sequence(DOUBLE), s60 sequence(DOUBLE), s61 sequence(DOUBLE), s62 sequence(DOUBLE), s63 sequence(DOUBLE), s64 sequence(DOUBLE), s65 sequence(DOUBLE), s66 sequence(DOUBLE), s67 sequence(DOUBLE), s68 sequence(DOUBLE), s69 sequence(DOUBLE), s70 sequence(DOUBLE), s71 sequence(DOUBLE), s72 sequence(DOUBLE), s73 sequence(DOUBLE), s74 sequence(DOUBLE), s75 sequence(DOUBLE), s76 sequence(DOUBLE), s77 sequence(DOUBLE), s78 sequence(DOUBLE), s79 sequence(DOUBLE), s80 sequence(DOUBLE), s81 sequence(DOUBLE), s82 sequence(DOUBLE), s83 sequence(DOUBLE), s84 sequence(DOUBLE), s85 sequence(DOUBLE), s86 sequence(DOUBLE), s87 sequence(DOUBLE), s88 sequence(DOUBLE), s89 sequence(DOUBLE), s90 sequence(DOUBLE), s91 sequence(DOUBLE), s92 sequence(DOUBLE), s93 sequence(DOUBLE), s94 sequence(DOUBLE), s95 sequence(DOUBLE), s96 sequence(DOUBLE), s97 sequence(DOUBLE), s98 sequence(DOUBLE), s99 sequence(DOUBLE));")
# curs.execute("trace on")
curs.execute("insert or update into d1_ingest_v select id_station,(substr(t,1,10)||' '||substr(t,12,8))::timestamp::bigint,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32,s33,s34,s35,s36,s37,s38,s39,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49,s50,s51,s52,s53,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63,s64,s65,s66,s67,s68,s69,s70,s71,s72,s73,s74,s75,s76,s77,s78,s79,s80,s81,s82,s83,s84,s85,s86,s87,s88,s89,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99 from foreign table(path='/home/abdel/d1_data/d1.csv', skip=1) as d1_h;")
        

In [ ]:
# insert data